
## Часть 2. Использование модели

**Цель:** отработать навыки адаптации готовых моделей для решения прикладной задачи на русском языке, а также создание небольших демо для задач.



### [2 балла] Добавить модель переводчика

У вас уже есть готовая модель, которая может по картинке отвечать на текстовые запросы к картинке. Ваша цель — обобщить эту модель на русский язык, добавив модель переводчик, которая будет переводить запрос на русском языке в запрос на английском языке и передавать его модели. За основу вы можете взять языковую модель (например, https://huggingface.co/Helsinki-NLP/opus-mt-ru-en). Альтернативой может стать реализация функции, делающий api вызов, к приложению переводчика (например, https://libretranslate.com/).

---

**Ожидаемый результат.** В качестве результата в этой секции вам нужно предоставить функции, которые делают перевод с русского на английского и делает инференс модели DocVQA и выводит ответ на русском языке. (В качестве примеров вопросов, можете использовать данные из датасета).


In [1]:
from transformers import MarianMTModel, MarianTokenizer
from datasets import load_dataset
from PIL import Image
import torch
from transformers import LayoutLMv2ForQuestionAnswering, LayoutLMv2Processor

C:\Users\Admin\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: Could not import module 'MarianMTModel'. Are this object's requirements defined correctly?

In [2]:
ru_en_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ru-en')
ru_en_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-ru-en')

en_ru_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-ru')
en_ru_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-ru')

device = "cuda" if torch.cuda.is_available() else "cpu"
ru_en_model.to(device)
en_ru_model.to(device)

def translate_ru2en(text):
    inputs = ru_en_tokenizer([text], return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    translated = ru_en_model.generate(**inputs)
    tgt_text = ru_en_tokenizer.batch_decode(translated, skip_special_tokens=True)[0]
    return tgt_text

def translate_en2ru(text):
    inputs = en_ru_tokenizer([text], return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    translated = en_ru_model.generate(**inputs)
    tgt_text = en_ru_tokenizer.batch_decode(translated, skip_special_tokens=True)[0]
    return tgt_text

# Загружаем зафайнтюненную модель и процессор LayoutLMv2
model_name = "Vendor62/finetuned-docvqa"
docvqa_model = LayoutLMv2ForQuestionAnswering.from_pretrained(model_name).to(device)
docvqa_processor = LayoutLMv2Processor.from_pretrained(model_name)

def docvqa_inference(question_en, model, processor, image, words, boxes):
    encoding = processor(image, words, boxes=boxes, text=question_en, return_tensors="pt")
    encoding = {k: v.to(device) for k, v in encoding.items()}
    outputs = model(**encoding)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    input_ids = encoding['input_ids'][0][answer_start:answer_end]
    answer = processor.tokenizer.decode(input_ids, skip_special_tokens=True)
    return answer

def ru_inference(question_ru, model, processor, image, words, boxes):
    question_en = translate_ru2en(question_ru)
    answer_en = docvqa_inference(question_en, model, processor, image, words, boxes)
    answer_ru = translate_en2ru(answer_en)
    return answer_ru

# Загрузка одного примера из датасета DocVQA
dataset = load_dataset("nielsr/docvqa_1200_examples", split="test[:1]")
example = dataset[0]

doc_image = example['image'].convert("RGB")
words = example['words']            # OCR слова
boxes = example['bounding_boxes']   # Бокс координаты

# Пример вопроса из датасета (на английском можно, но лучше на русском и переводить)
question_ru = "Как называется документ?"

# Запуск инференса
answer = ru_inference(question_ru, docvqa_model, docvqa_processor, doc_image, words, boxes)
print("Вопрос:", question_ru)
print("Ответ:", answer)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


ImportError: 
LayoutLMv2Model requires the detectron2 library but it was not found in your environment. Check out the instructions on the
installation page: https://github.com/facebookresearch/detectron2/blob/master/INSTALL.md and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [3]:
# Пример под PyTorch 2.1 и CUDA 11.7
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu117/torch2.1/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu117/torch2.1/index.html
ERROR: Could not find a version that satisfies the requirement detectron2 (from versions: none)
ERROR: No matching distribution found for detectron2


In [4]:
question_en = example['query']['en']
answer_en = docvqa_inference(question_en, docvqa_model, docvqa_processor, doc_image)
answer_ru = translate_en2ru(answer_en)

print("Вопрос (EN):", question_en)
print("Ответ (RU):", answer_ru)

/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:1621: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Вопрос (EN): What the location address of NSDA?
Ответ (RU): DA? Лучшая вещь между двумя бутербродами. Мягкие напитки идут со всеми видами сэндвичей. Круглые, квадратные, толстые, толстые, толстые. Они не только угощают большие жажды забавным способом, но и помогают бальзамировать диету. В конце концов здоровым телам нужно от 5 до 6 стаканов воды в день. Мягкие напитки содержат самую чистую, фильтрованную воду. Сэндвичи из ваших бутербродов. И празднуют национальный Сандвичевый месяц каждый месяц в году. Для информации о мягких напитках и сбалансированной диете, напишите: NAFT DRINK Association 1128 ШЕСТАТНЫЙ ST., N.W., WASATINGTON, D. 20036


### [2 балла] Сделать демо на gradio

Модель готова! Теперь было бы круто, если модель можно было захостить и оттестировать на практике. В этом задании вам нужно будет реализовать демо на gradio, которое будет принимать изображение и вопрос, а далее выдавать ответ. Пример демо, аналогично которому вам нужно реализовать модель: https://huggingface.co/spaces/nielsr/comparing-VQA-models.


**Подсказка:** во вкладке `Files` на демо вы можете посмотреть реализацию, там нужно заменить инференс, используемой модели, на инференс нашей модели с переводом


**Ожидаемый результат.** В качестве результата в этой секции вам нужно код для запуска демо на градио и видеозапись его работы, где реализован описанный выше функционал. Видео прикрепляйте отдельным файлом.

In [ ]:
#import nest_asyncio
#nest_asyncio.apply()

import gradio as gr
import torch
from transformers import MarianMTModel, MarianTokenizer
from transformers import LayoutLMv3ForQuestionAnswering, LayoutLMv3Processor

ru_en_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ru-en')
ru_en_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-ru-en')
en_ru_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-ru')
en_ru_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-ru')
model_name = 'microsoft/layoutlmv3-base'
docvqa_model = LayoutLMv3ForQuestionAnswering.from_pretrained(model_name)
docvqa_processor = LayoutLMv3Processor.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
docvqa_model.to(device)
ru_en_model.to(device)
en_ru_model.to(device)

def translate_ru2en(text):
    inputs = ru_en_tokenizer([text], return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    translated = ru_en_model.generate(**inputs)
    tgt_text = ru_en_tokenizer.batch_decode(translated, skip_special_tokens=True)[0]
    return tgt_text

def translate_en2ru(text):
    inputs = en_ru_tokenizer([text], return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    translated = en_ru_model.generate(**inputs)
    tgt_text = en_ru_tokenizer.batch_decode(translated, skip_special_tokens=True)[0]
    return tgt_text

def docvqa_inference(question_en, model, processor, image):
    encoding = processor(image, question_en, return_tensors="pt")
    encoding = {k: v.to(device) for k, v in encoding.items()}
    outputs = model(**encoding)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    input_ids = encoding['input_ids'][0][answer_start:answer_end]
    answer = processor.tokenizer.decode(input_ids, skip_special_tokens=True)
    return answer

def generate_answer_docvqa(image, question_ru):
    if image is None or not question_ru.strip():
        return "Пожалуйста, загрузите изображение и введите вопрос."
    question_en = translate_ru2en(question_ru)
    answer_en = docvqa_inference(question_en, docvqa_model, docvqa_processor, image)
    answer_ru = translate_en2ru(answer_en)
    return answer_ru

iface = gr.Interface(
    fn=generate_answer_docvqa,
    inputs=[gr.Image(type="pil"), gr.Textbox(label="Введите вопрос на русском")],
    outputs=gr.Textbox(label="Ответ"),
    title="DocVQA с переводом",
    description="Загрузите изображение документа и задайте вопрос на русском, получите ответ на русском."
)

In [ ]:
iface.launch(debug=True, prevent_thread_lock=True)

### [4 балла] Ответы на вопросы голосом

Демо готово! Но кто хочет писать вопросы текстом?
Здесь вам предстоить улучшить ваше демо, чтобы оно могло принимать вопросы голосом. За основу вам предлагается рассмотреть демо https://www.gradio.app/guides/real-time-speech-recognition и добавить соответствуещее окошко в ваше демо. Также вы можете добавить text-to-speech модель, чтобы оно озвучило текстовый ответ (дополнительный балл к оценке).

---

**Ожидаемый результат.** В качестве результата в этой секции вам нужно код для запуска демо на градио и видеозапись его работы, где реализован описанный выше функционал.

In [ ]:
# TODO Реализация демо на gradio с голосовым вводом